## 安裝 jieba 中文斷字工具

In [ ]:
!pip install jieba

## 測試中文斷字

In [ ]:
import jieba
import logging
jieba.setLogLevel(logging.INFO)

askList = '你的名字是甚麼?'.split()
print(askList)

## 中文版的自動回答機械人 ver#2

In [5]:
DB = [('你多大了?','我13嵗了.'),
      ('你還好嗎?','我很好.'),
      ('你吃早餐了嗎?','吃了.'),
      ('打游戲麽?','打.'),
      ('放學了嗎?','放了.'),
      ('你在做什麼?','我在和你聊天'),
      ('你知道我是誰嗎?','我不知道'),
      ('你想去哪裡玩嗎?','我想一直在這裡和你聊天'),
      ('你會一直和我聊天嗎?','我會的,我一直都在'),
      ('你有家嗎?在哪裡?','我的家就在這裡'),
      ('你喜歡和我聊天嗎?','當然喜歡,和你聊天是我的工作'),
      ('你是誰?','我會是你一直的陪伴者'),
      ('你會知道我的學校在哪嗎?','澳門高士德馬路7'),
      ('你喜歡打遊戲嗎？','喜歡'),
      ('你愛上學嗎？','還好'),
      ('你吃晚飯了嗎？?','吃了'),
      ('你喜歡喝七喜還是可樂？','七喜'),
      ('女生為什麼要化妝？','因為她們想將自己最好的一面展示給別人'),
      ('你有學過柔道嗎？','有'),
      ('你覺得學校應該招收多點英文老師嗎？','應該'),
      ('你追星嗎？','不'),
      ('你支持蜜糖陳皮嗎？','支持'),
      ('我們的班主任美嗎？','很美'),
      ('你今日食左咩?','蘋果'),
      ('你會說中文嗎？','會'),
      ('你的英文名字是什麼？','aiken'),
      ('你有幾個朋友？','三個'),
      ('你喜歡吃蘋果嗎？','喜歡'),
      ('你今天早上吃早餐了嗎?','吃了'),
      ('你的夢想是什麼？','警察'),
      ('你喜歡藍色嗎？','不'),
      ('你的名字是甚麼?','我的名字叫Ian.'),
      ('你今年幾多歲?','我今年14歲.'),
      ('PCMS的全稱是甚麼?','澳門培正中學.'),
      ('培正中學在哪裡?','澳門高士德馬路七號.'),
      ('幾點鐘開始上課?','8:05開始上課.'),
      ('你今天過得怎麼樣?','很好'),
      ('你在學校有多少朋友?','五個'),
      ('你中文考試滿分了嗎','不'),
      ('你能說英語嗎','是的'),
      ('現在幾點了?','三點'),
      ('你午餐打算吃什麼?','麵'),
      ('你家裡最喜歡誰?','父親'),
      ('你現在的工作是什麼?','消防隊員'),
      ('你的中文名字是什麼?','李甄善'),
      ('你是誰?','我是機械人.'),
      ('你在哪?','我在地球.'),
      ('你的早餐是什麼?','滑蛋叉燒三文治.'),
      ('你喜歡做什麼?','我喜歡看書.'),
      ('你喜歡什麼書?','武俠小說.'),
      ('你吃了早餐嗎?','吃了.'),
      ('你今年幾多歲?','我今年18歲.'),
      ('PCMS的全稱是甚麼?','澳門培正中學.'),
      ('你做什麼工作?','警察.'),
      ('你喜歡吃什麼?','巧克力.'),
      ('你有養寵物嗎?','有.'),
      ('你父親的工作是什麼?','醫生.'),
      ('你有兄弟姐妹嗎?','有.'),
      ('你喜歡打籃球嗎?','喜歡.'),
      ('你喜歡黃色嗎?','喜歡.'),
      ('你有學過鋼琴嗎？','有'),
      ('你喜歡你媽嗎？','還好'),
      ('英語課無聊還是語文課無聊？','語文課'),
      ('鴨子可愛還是鵝可愛？','鴨子'),
      ('你喜歡四川還是上海？','各有各的好，但我還是比較喜歡上海'),
      ('上學有什麼開心的地方？','可以認識很多的朋友'),
      ('數學AI和資訊科技的區別？','雖然兩個都是學AI，但AI也有很多種類，學的不一樣'),
      ('早飯吃了什麼？','今天早上吃了個營養麥片配豆漿'),
      ('午飯有什麼推薦的地方嗎？','如果最好能在家吃就在家吃，時間緊迫可以適當的選擇快餐店'),
      ('對於中文要演講有什麼想說的？','不是很想在同學面前演講'),
      ('你叫什麼名字？', '我叫團團'),
      ('你今年多少歲？', '我今年19歲'),
      ('你在哪裡上學？', '我不用上學'),
      ('你有什麼興趣？', '我沒有興趣'),
      ('你喜歡幹什麼？', '我喜歡睡覺'),
      ('上学', '我不用上学'),
      ('你會不會死?','不會'),
      ('討厭英文嗎?','還好'),
      ('你覺不覺得自己是個人?','是'),
      ('你有什麼事?','没有'),
      ('有没有空食藥?','不'),
      ('你喜不喜歡看抖音?','不'),
      ('喜不喜聽音樂?','嗯, 尤其是日文歌'),
      ('喜不喜廣東話?','嗯'),
      ('想不想打機?','等下, 跑完圈打'),
      ('你回家嗎?','我十點回'),
      ('你的名字是甚麼?','我的名字叫Harry.'),
      ('你今年幾多歲?','我今年18歲.'),
      ('PCMS的全稱是甚麼?','澳門培正中學.'),
      ('培正中學在哪裡?','澳門高士德馬路七號.'),
      ('幾點鐘開始上課?','8:05開始上課.'),
      ('你的主人是誰?','梁浩然'),
      ('你在邊間學校？','培正'),
      ('培正的全稱是什麼？','澳門培正中學'),
      ('住在邊到？','中國澳門'),
      ('你好嗎?','我很好'),
      ('你現在心情好嗎？','非常好，我很開心'),
      ('你喜歡做什麼？','拍攝。'),
      ('你有參加什麼課外活動嗎？','有，學日文。'),
      ('你喜歡吃什麼？','我喜歡吃蛋糕。'),
      ('你喜歡什麼動物？','我喜歡小貓。'),
      ('你長大後想當什麼？','我長大後想當一位Vtuber'),
      ('你喜歡食什麼零食?','朱古力.'),
      ('你好嗎?','我很好,不用擔心.'),
      ('你在那裡?','我在澳門培正中學.'),
      ('你是誰?','我是黃穎謙.'),
      ('你有沒有竉物?','我有一隻狗.'),
      ('你愛你的家人嗎?','非常愛.'),
      ('你讀那間學校?','培正.'),
      ('你有兄弟姐妹嗎?','有,一個姐姐.'),
      ('你喜歡什麼運動?','排球.'),
      ('你的是那一班?','正班.'),
      ('你叫什麼名字？', '我叫00413號電腦.'),
      ('你今年多少歲？', '我今天剛好28歲生日.'),
      ('你最愛吃哪一種水果？', '我喜歡吃蜜瓜'),
      ('你早飯吃了什麼？', '豆漿油條粢飯'),
      ('你最喜歡的城市是什麼？', '上海北京'),
      ('你最近聽的一首歌是什麼？', '想去海邊'),
      ('你最愛的電影是哪一部？', '楚門的世界.'),
      ('你喜歡貓還是喜歡狗？', '我更喜歡貓.'),
      ('你喜歡密室逃脫還是劇本殺？', '我更喜歡劇本殺.'),
      ('你的興趣愛好是什麼', '我最愛唱歌跳舞.'),
      ('加工费', '100000'),
      ('ui很久很久', '9000000'),
      ('你好嗎？','我很好'),
      ('你在哪裏？','我在培正中學'),
      ('你多大？','我13嵗'),
      ('你會説什麽語言？','我會説中文和英文'),
      ('你吃了東西了嗎？','我吃了東西'),
      ('今天是不是你值日生？','不是'),
      ('你上課無不無聊？','無聊'),
      ('你住什麽城市？','澳門'),
      ('學校測驗難還是不難？','很難'),
      ('你喜歡打游戲嗎？', '喜歡'),
      ('語言', '我會説中文和英文'),
      ('說', '我會説中文和英文'),
      ('難不難', '很難'),
      ('多大', '我13嵗'),
      ('你喜歡吃什麼?','我喜歡吃白飯.'),
      ('你叫什麼名字?','我叫小美.'),
      ('你吃了早餐未?','我吃了.'),
      ('你喜歡上英文課嗎?','我不喜歡.'),
      ('你喜歡看動漫嗎?','不喜歡.'),
      ('你今天幾點起床?','6:45a.m.'),
      ('你喜歡喝珍珠奶茶嗎?','我喜歡'),
      ('你喜歡什麼顏色?',''),
	  ('顏色', '藍色'),
('歌', '歲月神偷'),
('全班人數', '42人'),
('平均身高', '150-160'),
('心情好嗎', '心情很好'),

   ('開心?','I am very happy now.'),
      ('今日中午食咩?','我今日中午食黃瓜Cucumber.'),
      ('hi?','Hello, nice to meet you.'),
      ('你知唔知我係邊個?','我係小朋友.'),
	  ('早餐', '沒吃'),
('午餐', '沒吃'),
('晚餐', '還沒吃'),
('記得付錢喔,親', '不要呢,寶'),
('下午茶', '不吃'),
('性別', '男生'),
('喜好', '唱歌'),
('放學時間', '12:15'),
('校長', '高校長'),
('飲料', '珍珠奶茶'),
  ('你會和我說再見嗎？','我們再不講再見'),
      ('但願你愛我，你回應一下好嗎？','我愛你，不過你不會知道編程的人是誰'),
      ('如果分手點算好？','我在這邊 陪你寂寞'),
      ('Collar定Mirror好D？','BLACKPINK'),
      ('你有咩問題？','我沒問題，你泰國人？？？'),
      ('魔鏡魔鏡，誰是培正的男神？','趙樂旻'),
	  ('早上好', '早上好哦'),
('初二什麽時候畢業', '2026'),
('晚上好', '你吃晚飯了嗎？'),
('巧克力很甜', '也有苦的好嗎'),
('希露菲超可愛的', '對啊'),
('世界有霍格沃茲嗎', '并沒有'),
('疫情什麽時候結束', '大概几年后吧'),
 ('你的偶像是誰?','我的偶像是xxx.'),
      ('你是女還是男?','我是女的.'),
      ('你喜歡食咩?','我喜歡食零食.'),
      ('你最喜歡咩課?','我最喜歡數學'),
      ('你好.','你好.'),
      ('再見.','再見.'),
  ('你喜欢吃什么？','我喜欢吃酸菜鱼'),
      ('你喜欢做什么运动？','我喜欢打篮球'),
      ('你喜欢做什么？','我喜欢看书'),
      ('今天心情如何？','我很开心'),
      ('你会做什么？','回答问题'),
      ('你食左飯未?','食左'),
      ('你的朋友是甚麼名字?','我的朋友叫Tommy'),
      ('你喜歡甚麼鞋?','Nike air fox 1'),
      ('你是男是?', '男'),
('有冇測驗', '冇'),
('中午放學', '12點'),
('校服顏色', '白色'),
('几高', '150'),
('天氣如何', '好'),
('培正那一年創校？', '1889年'),
      ('你午飯吃什麼 ?','米飯.'),
      ('你好','你好'),
      ('你開心嗎','開心'),
      ('你知道我是誰嗎?','不知道'),
      ('你結婚了嗎','沒有'),
      ('你是男是女','我是機器人'),
	     ('甚麼時候放暑假？', '2022 年 7 月 2 '),
      ('昨天晚飯吃了甚麼？', '米飯、菜，肉'),
      ('補習班甚麼時候開始？', '17:00'),
      ('澳門培正中學現任校長是誰？','高錦輝校長'),
	     ('幾點鐘下課?','3:45下課.'),
      ('你吃飯了嗎?','我吃飯了.'),
      ('今天天氣如何?','今天天氣很好.'),
      ('你會談戀愛嗎?','我不懂這些.'),
      ('你談過戀愛嗎?','我沒有談過戀愛.'),
      ('你的性別是什麼?','我的性別是女.'),
	  ('你偶像是誰?', '我偶像是BLACKPINK.'),
('噗噗的主人是誰?', '噗噗的主人是羽生結弦.'),
('今日中午食咩', '食飯團'),
 ('食飯了嗎?','還沒有.'),
      ('你在那學習?','澳門培正中學.'), 
      ('幾點放學?','12:15.'),
      ('你還好嗎?','很好.'),
      ('再見','byebye'),
	  ('pcms', '澳門培正中學'),
('開不開心', '開心'),
('食飯了嗎', '還未'),
('最喜歡吃什麼', '巧克力'),
('今天天氣', '晴天'),
('几点下課', '12:15'),
('你今天要吃什麼？','我想吃猪扒包.'),
      ('你喜歡做什麼啊？','我喜歡畫畫和跳舞.'),
      ('你想去哪裡度假?','我想去日本或加拿大度假.'),
      ('今天應該去哪裡呢？','我們去遊樂園玩吧!'),
      ('我現在要睡覺了晚安.','好吧晚安.'),
	  ('1+1', '2'),
('b22位置', '王彥智靚仔的位置'),
('幹什麼', '吃飯中'),
('笑話', '有一天有個帥哥走在路上  一個阿嬤突然上前搭訕說「帥哥，你超會搭耶」  然後帥哥就冒煙了  (台語 : 超會搭 = 烤焦了)'),
('返學意義', '1、想成功就要学习，我们要成功是不是要学习那学习；一种是使用别人证明有效的方法；2、节省时间看完一本书需要三天，通过看书只能学到；课程已经有一套总结完毕的方法跟系统；上课就是快速给你一套总结完毕的方法跟系统，所以课；3、学习要来上课假如今天一个人不来学习，在外面用；他不来上课，他用摸索的方法，损失了很多钱；可是上课只要几千元或几百'),
('下午放學', '3:45'),
('加油', '加油!'),
('幾點落堂？', '12:15a.m.和15:45p.m.'),
('You are so smart', 'Thank you'),
('幾時測驗','11;05'),
('你幾時做功課','12;00'),
     ('可以幫我做生意','當然可以'),
	 ('你是不是好人','是'),
	 ('你會不會幫人','會'),
	       ('1+1?','2'),
      ('1+2?','3'),
      ('1+3?','4'),
      ('1+4?','5'),
      ('1+5?','6'),
	  ('你在做什麼?', '吃飯中.'),
('上課', '上課中'),
('下課', '12:00'),
('做什麼', '食飯中'),
('如何提高成績', '努力温習'),
('怎樣買手機', '用錢買'),
('朋友', '兩個人的友誼'),
('什么时候吃饭?','12.30'),
      ('什么时候吃早餐','9.00'),
	    ('你是哪個國家的?','中國,澳門特別行政區'),
      ('今天怎麼樣?','還好，老實說其實不是很好'),
      ('今天有什麼活動?','今天下午去游泳，然後八點吃完晚飯和家人去湖邊散步.'),
	  ('聽日有咩做?','我希望冇嘢做'),
     ('你的夢想是甚麼?','我希望成為同我遠房親戚SIRI一樣偉大的人工智能'),
     ('你識唔識做後空翻','如果給我一副身體,我想我可以'),
     ('點解你咁傻','因為你係咁教我囉'),
     ('起咗身未?','俾你弄醒了'),
	 ('下課', '12:05'),
('下午上課', '14:15'),
('課室在哪', 'C303'),
('食飯未', '未'),
('2+2=什麼', '4'),
('今天中午吃什麽?','吃你想吃的.'),
               ('沒錢了怎麽辦?','找爸媽.'),
               ('待會兒考試我沒復習怎麽辦?','隨機殺死一名同學進入監獄后申請緩考.'),
			   ('你最喜欢什么课堂？','中文'),
      ('是什么老师教你中文的?','容sir'),
      ('你们什么是后放学?','3：45'),
      ('放学以后你干什么？','去麦当劳'),
      ('你中午去哪吃？','麦当劳'),
      ('你有没有喜欢的人','没有'),
	  ('你幾點下課?','3:45下課.'),
      ('你温左習未','已經温完'),
	  ('課室在哪', '在C303'),
('我很不開心', '沒事，一切會好起來的'),
('我想去吃飯', '去M記吧'),
('我想去運動', '打籃球吧'),
('我想去學編程', '去找施sir學'),
('hello', 'hello'),
('下課', '12:15'),
('夢想', 'find one piece'),
('聽日有咩做', '聽日冇野做'),
('放假', '聽日放假'),
('最后一节什么时候下课?','12:15下课'),
      ('吃早餐了吗?','吃了薄饼'),
      ('等一下吃什么?','吃意大利面'),
      ('要不要一起吃饭?','不了等一下有人找我')]
	  
	  

In [6]:
import jieba
import logging
jieba.setLogLevel(logging.INFO)


def match(alist,blist):
    cout = 0
    for a in alist:
        if a in blist:
            cout = cout+1
    return cout 

def printDB():
    print('DB=[')
    for i in range(len(DB)-1):
        print(f'{DB[i]},')
    print(DB[len(DB)-1])    
    print(']') 

while (True):
    youAsk = input('你問: ')
    
    if youAsk=='再見':
        printDB()
        break
    else:
        askList = jieba.lcut(youAsk)
        nlist = []
        for q,a in DB:
            qList = jieba.lcut(q)
            c = match(askList,qList)
            nlist.append((c,a))
              
        nlist = sorted(nlist,reverse=True)
        c,a = nlist[0]
        if (c==0):
            new_answer=input('資料庫找不到對應的回答, 請教我如何回答:')
            DB.append((youAsk,new_answer))
        else:    
            print('機械人:',a)

你問: 再見
DB=[
('你多大了?', '我13嵗了.'),
('你還好嗎?', '我很好.'),
('你吃早餐了嗎?', '吃了.'),
('打游戲麽?', '打.'),
('放學了嗎?', '放了.'),
('你在做什麼?', '我在和你聊天'),
('你知道我是誰嗎?', '我不知道'),
('你想去哪裡玩嗎?', '我想一直在這裡和你聊天'),
('你會一直和我聊天嗎?', '我會的,我一直都在'),
('你有家嗎?在哪裡?', '我的家就在這裡'),
('你喜歡和我聊天嗎?', '當然喜歡,和你聊天是我的工作'),
('你是誰?', '我會是你一直的陪伴者'),
('你會知道我的學校在哪嗎?', '澳門高士德馬路7'),
('你喜歡打遊戲嗎？', '喜歡'),
('你愛上學嗎？', '還好'),
('你吃晚飯了嗎？?', '吃了'),
('你喜歡喝七喜還是可樂？', '七喜'),
('女生為什麼要化妝？', '因為她們想將自己最好的一面展示給別人'),
('你有學過柔道嗎？', '有'),
('你覺得學校應該招收多點英文老師嗎？', '應該'),
('你追星嗎？', '不'),
('你支持蜜糖陳皮嗎？', '支持'),
('我們的班主任美嗎？', '很美'),
('你今日食左咩?', '蘋果'),
('你會說中文嗎？', '會'),
('你的英文名字是什麼？', 'aiken'),
('你有幾個朋友？', '三個'),
('你喜歡吃蘋果嗎？', '喜歡'),
('你今天早上吃早餐了嗎?', '吃了'),
('你的夢想是什麼？', '警察'),
('你喜歡藍色嗎？', '不'),
('你的名字是甚麼?', '我的名字叫Ian.'),
('你今年幾多歲?', '我今年14歲.'),
('PCMS的全稱是甚麼?', '澳門培正中學.'),
('培正中學在哪裡?', '澳門高士德馬路七號.'),
('幾點鐘開始上課?', '8:05開始上課.'),
('你今天過得怎麼樣?', '很好'),
('你在學校有多少朋友?', '五個'),
('你中文考試滿分了嗎', '不'),
('你能說英語嗎', '是的'),
('現在幾點了?', '三點'),
('你午餐打算吃什麼?', '麵'),
('你家裡最喜歡誰?', '父親'),
('你現在的工作是什麼?', '消防隊

## 中文版的自動回答機械人 ver#3

In [1]:
DB = [('你多大了?','我13嵗了.'),
      ('你還好嗎?','我很好.'),
      ('你吃早餐了嗎?','吃了.'),
      ('打游戲麽?','打.'),
      ('放學了嗎?','放了.'),
      ('你在做什麼?','我在和你聊天'),
      ('你知道我是誰嗎?','我不知道'),
      ('你想去哪裡玩嗎?','我想一直在這裡和你聊天'),
      ('你會一直和我聊天嗎?','我會的,我一直都在'),
      ('你有家嗎?在哪裡?','我的家就在這裡'),
      ('你喜歡和我聊天嗎?','當然喜歡,和你聊天是我的工作'),
      ('你是誰?','我會是你一直的陪伴者'),
      ('你會知道我的學校在哪嗎?','澳門高士德馬路7'),
      ('你喜歡打遊戲嗎？','喜歡'),
      ('你愛上學嗎？','還好'),
      ('你吃晚飯了嗎？?','吃了'),
      ('你喜歡喝七喜還是可樂？','七喜'),
      ('女生為什麼要化妝？','因為她們想將自己最好的一面展示給別人'),
      ('你有學過柔道嗎？','有'),
      ('你覺得學校應該招收多點英文老師嗎？','應該'),
      ('你追星嗎？','不'),
      ('你支持蜜糖陳皮嗎？','支持'),
      ('我們的班主任美嗎？','很美'),
      ('你今日食左咩?','蘋果'),
      ('你會說中文嗎？','會'),
      ('你的英文名字是什麼？','aiken'),
      ('你有幾個朋友？','三個'),
      ('你喜歡吃蘋果嗎？','喜歡'),
      ('你今天早上吃早餐了嗎?','吃了'),
      ('你的夢想是什麼？','警察'),
      ('你喜歡藍色嗎？','不'),
      ('你的名字是甚麼?','我的名字叫Ian.'),
      ('你今年幾多歲?','我今年14歲.'),
      ('PCMS的全稱是甚麼?','澳門培正中學.'),
      ('培正中學在哪裡?','澳門高士德馬路七號.'),
      ('幾點鐘開始上課?','8:05開始上課.'),
      ('你今天過得怎麼樣?','很好'),
      ('你在學校有多少朋友?','五個'),
      ('你中文考試滿分了嗎','不'),
      ('你能說英語嗎','是的'),
      ('現在幾點了?','三點'),
      ('你午餐打算吃什麼?','麵'),
      ('你家裡最喜歡誰?','父親'),
      ('你現在的工作是什麼?','消防隊員'),
      ('你的中文名字是什麼?','李甄善'),
      ('你是誰?','我是機械人.'),
      ('你在哪?','我在地球.'),
      ('你的早餐是什麼?','滑蛋叉燒三文治.'),
      ('你喜歡做什麼?','我喜歡看書.'),
      ('你喜歡什麼書?','武俠小說.'),
      ('你吃了早餐嗎?','吃了.'),
      ('你今年幾多歲?','我今年18歲.'),
      ('PCMS的全稱是甚麼?','澳門培正中學.'),
      ('你做什麼工作?','警察.'),
      ('你喜歡吃什麼?','巧克力.'),
      ('你有養寵物嗎?','有.'),
      ('你父親的工作是什麼?','醫生.'),
      ('你有兄弟姐妹嗎?','有.'),
      ('你喜歡打籃球嗎?','喜歡.'),
      ('你喜歡黃色嗎?','喜歡.'),
      ('你有學過鋼琴嗎？','有'),
      ('你喜歡你媽嗎？','還好'),
      ('英語課無聊還是語文課無聊？','語文課'),
      ('鴨子可愛還是鵝可愛？','鴨子'),
      ('你喜歡四川還是上海？','各有各的好，但我還是比較喜歡上海'),
      ('上學有什麼開心的地方？','可以認識很多的朋友'),
      ('數學AI和資訊科技的區別？','雖然兩個都是學AI，但AI也有很多種類，學的不一樣'),
      ('早飯吃了什麼？','今天早上吃了個營養麥片配豆漿'),
      ('午飯有什麼推薦的地方嗎？','如果最好能在家吃就在家吃，時間緊迫可以適當的選擇快餐店'),
      ('對於中文要演講有什麼想說的？','不是很想在同學面前演講'),
      ('你叫什麼名字？', '我叫團團'),
      ('你今年多少歲？', '我今年19歲'),
      ('你在哪裡上學？', '我不用上學'),
      ('你有什麼興趣？', '我沒有興趣'),
      ('你喜歡幹什麼？', '我喜歡睡覺'),
      ('上学', '我不用上学'),
      ('你會不會死?','不會'),
      ('討厭英文嗎?','還好'),
      ('你覺不覺得自己是個人?','是'),
      ('你有什麼事?','没有'),
      ('有没有空食藥?','不'),
      ('你喜不喜歡看抖音?','不'),
      ('喜不喜聽音樂?','嗯, 尤其是日文歌'),
      ('喜不喜廣東話?','嗯'),
      ('想不想打機?','等下, 跑完圈打'),
      ('你回家嗎?','我十點回'),
      ('你的名字是甚麼?','我的名字叫Harry.'),
      ('你今年幾多歲?','我今年18歲.'),
      ('PCMS的全稱是甚麼?','澳門培正中學.'),
      ('培正中學在哪裡?','澳門高士德馬路七號.'),
      ('幾點鐘開始上課?','8:05開始上課.'),
      ('你的主人是誰?','梁浩然'),
      ('你在邊間學校？','培正'),
      ('培正的全稱是什麼？','澳門培正中學'),
      ('住在邊到？','中國澳門'),
      ('你好嗎?','我很好'),
      ('你現在心情好嗎？','非常好，我很開心'),
      ('你喜歡做什麼？','拍攝。'),
      ('你有參加什麼課外活動嗎？','有，學日文。'),
      ('你喜歡吃什麼？','我喜歡吃蛋糕。'),
      ('你喜歡什麼動物？','我喜歡小貓。'),
      ('你長大後想當什麼？','我長大後想當一位Vtuber'),
      ('你喜歡食什麼零食?','朱古力.'),
      ('你好嗎?','我很好,不用擔心.'),
      ('你在那裡?','我在澳門培正中學.'),
      ('你是誰?','我是黃穎謙.'),
      ('你有沒有竉物?','我有一隻狗.'),
      ('你愛你的家人嗎?','非常愛.'),
      ('你讀那間學校?','培正.'),
      ('你有兄弟姐妹嗎?','有,一個姐姐.'),
      ('你喜歡什麼運動?','排球.'),
      ('你的是那一班?','正班.'),
      ('你叫什麼名字？', '我叫00413號電腦.'),
      ('你今年多少歲？', '我今天剛好28歲生日.'),
      ('你最愛吃哪一種水果？', '我喜歡吃蜜瓜'),
      ('你早飯吃了什麼？', '豆漿油條粢飯'),
      ('你最喜歡的城市是什麼？', '上海北京'),
      ('你最近聽的一首歌是什麼？', '想去海邊'),
      ('你最愛的電影是哪一部？', '楚門的世界.'),
      ('你喜歡貓還是喜歡狗？', '我更喜歡貓.'),
      ('你喜歡密室逃脫還是劇本殺？', '我更喜歡劇本殺.'),
      ('你的興趣愛好是什麼', '我最愛唱歌跳舞.'),
      ('加工费', '100000'),
      ('ui很久很久', '9000000'),
      ('你好嗎？','我很好'),
      ('你在哪裏？','我在培正中學'),
      ('你多大？','我13嵗'),
      ('你會説什麽語言？','我會説中文和英文'),
      ('你吃了東西了嗎？','我吃了東西'),
      ('今天是不是你值日生？','不是'),
      ('你上課無不無聊？','無聊'),
      ('你住什麽城市？','澳門'),
      ('學校測驗難還是不難？','很難'),
      ('你喜歡打游戲嗎？', '喜歡'),
      ('語言', '我會説中文和英文'),
      ('說', '我會説中文和英文'),
      ('難不難', '很難'),
      ('多大', '我13嵗'),
      ('你喜歡吃什麼?','我喜歡吃白飯.'),
      ('你叫什麼名字?','我叫小美.'),
      ('你吃了早餐未?','我吃了.'),
      ('你喜歡上英文課嗎?','我不喜歡.'),
      ('你喜歡看動漫嗎?','不喜歡.'),
      ('你今天幾點起床?','6:45a.m.'),
      ('你喜歡喝珍珠奶茶嗎?','我喜歡'),
      ('你喜歡什麼顏色?','')]

In [2]:
from flask import Flask, request, send_from_directory
from ipywidgets import widgets
from IPython.display import display
import io
import jieba, json



def match(alist,blist):
    cout = 0
    for a in alist:
        if a in blist:
            cout = cout+1
    return cout 

def printDB():
    print('DB=[')
    for i in range(len(DB)-1):
        print(f'{DB[i]},')
    print(DB[len(DB)-1])    
    print(']') 

app = Flask("ai chat 7")


@app.route("/")
def form():
    with open('chatbot.html', 'r',encoding='utf8') as chatbotfile:
        data = chatbotfile.read()
    return data

    
@app.route("/ask", methods=['POST', 'GET'])
def ask():
    if not request.args.get('text'):
        return '都沒有文字:<';
    
    youAsk = request.args.get('text')
    print('有人問: %s'%youAsk)
    
    askList = jieba.lcut(youAsk)
    nlist = []
    for q,a in DB:
        qList = jieba.lcut(q)
        c = match(askList,qList)
        nlist.append((c,a))

    nlist = sorted(nlist,reverse=True)
    c,a = nlist[0]
    if (c==0):
        new_answer=input('資料庫找不到對應的回答, 請教我如何回答:')
        DB.append((youAsk,new_answer))
        response = new_answer
    else:    
        response = a
   
    print('機械人:',response)
    return response



try: 
    # works fine while running the py script in the command-line 
    app.run(debug=True,port=8000)
except:
    # internal issue with Jupyter Notebook
    print("Exception occured!")
    # running manually
    from werkzeug.serving import run_simple
    run_simple('localhost', 8000, app)

Exception occured!


 * Running on http://localhost:8000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Apr/2022 18:02:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 18:02:22] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [19/Apr/2022 18:02:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 18:02:40] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [19/Apr/2022 18:18:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 18:18:28] "GET /favicon.ico HTTP/1.1" 404 -
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ricky\AppData\Local\Temp\jieba.cache


有人問: Hi


Loading model cost 0.807 seconds.
Prefix dict has been built successfully.


資料庫找不到對應的回答, 請教我如何回答:不要了


127.0.0.1 - - [19/Apr/2022 18:18:48] "GET /ask?text=Hi HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 18:18:48] "GET /static/typing2.gif HTTP/1.1" 200 -


機械人: 不要了
